In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from tqdm import tqdm
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
review_path = "yelp_Fall2023/review.json"
business_path = "yelp_Fall2023/business.json"
mobility_path = "yelp_Fall2023/Trips_by_Distance.csv"

In [3]:
business_data = pd.read_json(business_path, lines = True)

In [4]:
business_data = business_data.dropna()

In [5]:
bars_business_data = business_data[business_data.categories.str.contains("bar|Bars")]

In [6]:
bars_business_data.describe()

,latitude,longitude,stars,review_count,is_open
count,12872.000000,12872.000000,12872.000000,12872.000000,12872.000000
mean,36.384210,-88.767756,3.715701,129.403745,0.701445
std,6.054382,13.839064,0.671914,221.384225,0.457642
min,27.555127,-119.983922,1.000000,5.000000,0.000000
25%,29.962553,-90.249359,3.500000,22.000000,0.000000
50%,38.627015,-86.146093,4.000000,60.000000,1.000000
75%,39.948843,-75.606171,4.000000,150.000000,1.000000
max,53.646365,-74.682591,5.000000,5721.000000,1.000000


In [7]:
business_data.describe()

,latitude,longitude,stars,review_count,is_open
count,117618.000000,117618.000000,117618.000000,117618.000000,117618.000000
mean,36.612308,-89.277679,3.659321,52.958238,0.807495
std,5.838800,14.804658,0.932171,135.126694,0.394269
min,27.555127,-120.095137,1.000000,5.000000,0.000000
25%,32.173332,-90.349720,3.000000,9.000000,1.000000
50%,38.731374,-86.120175,4.000000,18.000000,1.000000
75%,39.953499,-75.449811,4.500000,46.000000,1.000000
max,53.651838,-73.200457,5.000000,7568.000000,1.000000


In [8]:
business_data.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

In [9]:
# Define the criteria for successful business
bars_business_data['successful'] = ((bars_business_data['stars'] >= 3.5) & (bars_business_data['review_count'] >= 100)).astype('int64')

# Prepare the data for logistic regression
X = bars_business_data[['successful']]  # Predictor variable
y = bars_business_data['is_open']       # Response variable

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Perform logistic regression
model = sm.Logit(y, X)
result = model.fit()

# Print the summary of the regression
print(result.summary())

# Calculating the odds ratio
odds_ratio = pd.DataFrame({'Odds Ratio': result.params.apply(lambda x: np.exp(x))})
print(odds_ratio)

Optimization terminated successfully.
         Current function value: 0.593301
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                is_open   No. Observations:                12872
Model:                          Logit   Df Residuals:                    12870
Method:                           MLE   Df Model:                            1
Date:                Sun, 03 Dec 2023   Pseudo R-squ.:                 0.02679
Time:                        16:00:03   Log-Likelihood:                -7637.0
converged:                       True   LL-Null:                       -7847.2
Covariance Type:            nonrobust   LLR p-value:                 1.902e-93
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6105      0.022     27.541      0.000       0.567       0.654
successful     0.9196      0.

/var/folders/dy/d8nzqt0x6dlbq5kqh3ppwvs00000gn/T/ipykernel_15339/3864789580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bars_business_data['successful'] = ((bars_business_data['stars'] >= 3.5) & (bars_business_data['review_count'] >= 100)).astype('int64')


In [10]:
# illinois_business_data = 
temp = bars_business_data[bars_business_data.state == 'IL']

In [11]:
len(bars_business_data[(bars_business_data['stars'] >= 3.5) & (bars_business_data['review_count'] >= 100)])

3950

In [12]:
trips_data = pd.read_csv(mobility_path, index_col = None)

In [13]:
trips_data.tail()

,Level,Date,State FIPS,State Postal Code,County FIPS,County Name,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,...,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500,Row ID,Week,Month
5560749,State,2023/10/07,10.0,DE,NaN,NaN,193623.0,780141.0,4222025.0,1105040.0,...,693535.0,652105.0,223670.0,77334.0,26266.0,2843.0,2098.0,10-00000-20231007,40,10
5560750,State,2023/10/07,23.0,ME,NaN,NaN,265741.0,1078471.0,6422140.0,1475329.0,...,1058192.0,1117974.0,404352.0,119612.0,32754.0,2194.0,2987.0,23-00000-20231007,40,10
5560751,State,2023/10/07,40.0,OK,NaN,NaN,746731.0,3210240.0,16949155.0,4344331.0,...,2591151.0,2720144.0,952152.0,433461.0,201816.0,23013.0,7480.0,40-00000-20231007,40,10
5560752,State,2023/10/07,55.0,WI,NaN,NaN,1107713.0,4714721.0,28209395.0,7574840.0,...,4402765.0,4301354.0,1513604.0,611522.0,276734.0,30407.0,16301.0,55-00000-20231007,40,10
5560753,State,2023/10/07,54.0,WV,NaN,NaN,371816.0,1420331.0,7596814.0,1812012.0,...,1242039.0,1202927.0,430595.0,159008.0,67406.0,7926.0,1783.0,54-00000-20231007,40,10


In [14]:
trips_data.Level.value_counts()

Level
County      5470222
State         88791
National       1741
Name: count, dtype: int64

In [15]:
bars_business_data.is_open.value_counts()

is_open
1    9029
0    3843
Name: count, dtype: int64

In [16]:
state_data = trips_data[trips_data['Level'] == 'State']

# Convert 'Date' to datetime and extract the day of the week
state_data['DayOfWeek'] = pd.to_datetime(state_data['Date']).dt.day_name()

# Calculate average mobility (Number of Trips) for each state and day
average_mobility = state_data.groupby(['State Postal Code', 'DayOfWeek'])['Number of Trips'].mean()

# Identify top 3 days of mobility for each state
top_days = average_mobility.groupby(level=0).nlargest(3).reset_index(level=0, drop=True)

# Sample restaurant hours (assuming this format for each restaurant)
restaurant_hours = {
    'Tuesday': '0:0-0:0',
    'Wednesday': '13:30-22:0',
    'Thursday': '13:30-22:0',
    'Friday': '13:30-23:0',
    'Saturday': '13:30-23:0',
    'Sunday': '13:30-22:0'
}

# Function to check if a restaurant is open on a given day
def is_open_on_day(day, hours):
    if (day in hours):
        if (hours[day] != '0:0-0:0'):
            return True
        else:
            False
    return False

# Check if the restaurant is open on top mobility days
# for state, days in top_days.groupby('State Postal Code'):
#     print(f"State: {state}")
#     for day in days.index.get_level_values(1):
#         open_status = is_open_on_day(day, restaurant_hours)
#         print(f" - {day}: {'Open' if open_status else 'Closed'}")

/var/folders/dy/d8nzqt0x6dlbq5kqh3ppwvs00000gn/T/ipykernel_15339/1127205767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_data['DayOfWeek'] = pd.to_datetime(state_data['Date']).dt.day_name()


In [17]:
tdict = {}
for state, days in top_days.groupby('State Postal Code'):
    if state not in tdict:
        tdict[state] = np.array(days.index.get_level_values(1))

In [18]:
bars_business_data[bars_business_data['state'] != 'AB'].apply(
    lambda x: is_open_on_day(x['hours'], tdict[x['state']]), axis=1
)
bars_opdata = bars_business_data[['business_id', 'state', 'attributes', 'hours']].reset_index(drop = True)

In [19]:
bars_opdata['Monday'] = bars_opdata.hours.apply(lambda x : is_open_on_day("Monday", x))
bars_opdata['Tuesday'] = bars_opdata.hours.apply(lambda x : is_open_on_day("Tuesday", x))
bars_opdata['Wednesday'] = bars_opdata.hours.apply(lambda x : is_open_on_day("Wednesday", x))
bars_opdata['Thursday'] = bars_opdata.hours.apply(lambda x : is_open_on_day("Thursday", x))
bars_opdata['Friday'] = bars_opdata.hours.apply(lambda x : is_open_on_day("Friday", x))
bars_opdata['Saturday'] = bars_opdata.hours.apply(lambda x : is_open_on_day("Saturday", x))
bars_opdata['Sunday'] = bars_opdata.hours.apply(lambda x : is_open_on_day("Sunday", x))

In [20]:
del bars_opdata['hours']

In [23]:
bars_opdata = bars_opdata[bars_opdata['state'] != 'AB']

In [24]:
df = state_data.groupby(['State Postal Code', 'DayOfWeek'], as_index = False)['Number of Trips'].agg('mean')
max_sales = df.groupby('State Postal Code')['Number of Trips'].max()
min_sales = df.groupby('State Postal Code')['Number of Trips'].min()

# Calculating the difference and percentage difference
difference = max_sales - min_sales
percentage_difference = (difference / max_sales) * 100

# Creating a new DataFrame to display the results
result = pd.DataFrame({
    'Max Number of Trips': max_sales,
    'Min Number of Trips': min_sales,
    'Difference': difference,
    'Percentage Difference': percentage_difference
})

In [25]:
business_ids = bars_opdata.business_id.unique()
review_data = pd.DataFrame()
for chunk in tqdm(pd.read_json('yelp_Fall2023/review.json', lines = True, chunksize = 100000)):
    temp = chunk[chunk.business_id.isin(business_ids)]
    review_data = pd.concat([review_data, temp])

70it [03:27,  2.96s/it]


In [26]:
del review_data['user_id']
del review_data['useful']
del review_data['funny']
del review_data['cool']

In [29]:
tokenizer = AutoTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')
model = AutoModelForSeq2SeqLM.from_pretrained('sshleifer/distilbart-cnn-12-6')
def summarize_text(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=2, max_length=200, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [30]:
from multiprocessing.dummy import Pool as ThreadPool
long_reviews = review_data[review_data['text'].apply(lambda x: len(x.split(" ")) >= 100)].reset_index(drop=True)
# Function to process the reviews using ThreadPool
def process_reviews(reviews):
    pool = ThreadPool(4)  # Number of threads
    results = pool.map(summarize_text, reviews)
    pool.close()
    pool.join()
    return results

# Call the function and assign the results to the 'summary' column
long_reviews['summary'] = process_reviews(long_reviews['text'])


KeyboardInterrupt



In [31]:
long_reviews

,review_id,business_id,stars,text,date
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,LnGZB0fjfgeVDVz5IHuEVA,rBdG_23USc7DletfZ11xGA,4,The hubby and I have been here on multiple occ...,2014-08-10 19:41:43
2,ZVvhc3Go7v5I8XTiVoWmqQ,ut6fi2W2YaipNOqvi7e0jw,3,"Upland is a brewery based out of Bloomington, ...",2014-11-12 14:12:20
3,lUUhg8ltDsUZ9h0xnwY4Dg,cPepkJeRMtHapc_b2Oe_dw,4,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07
4,YcLXh-3UC9y6YFAI9xxzPQ,oBhJuukGRqPVvYBfTkhuZA,4,The only reason I didn't give this restaurant ...,2015-03-05 03:37:54
...,...,...,...,...,...
632074,4ncWtKqNnGcpHk2isnWZzg,IoRTm7IAJqBbHpiisUBnLg,5,I have been a regular of Tony's and the previo...,2020-07-30 18:36:12
632075,flql0FgydC3n0Hwx6kcJoQ,oHvEgLH6pAkcrPmeR1l3UQ,5,We are huge Retreat fans! One of my favorite t...,2018-04-26 05:15:22
632076,ZcEx4UEnTnR_TEPEqwkKjA,Ea663rIHyKXz2VP2DPH7Cg,4,I decided to try this place out after Christma...,2020-01-13 04:21:38
632077,pJvL9AN6gDoixiLLNF1c7w,r2IhvKZQ_wLR5mLBnPOilg,4,One of my favorite restaurants in Santa Barbar...,2019-03-27 04:16:27


In [33]:
# tdict

In [35]:
del bars_opdata['attributes']

In [37]:
bars_opdata.to_csv("../Documents/STAT628/bars_day_op_data.csv", index = None)

In [42]:
day = pd.DataFrame(tdict).T
day['state'] = day.index
day.to_csv("../Documents/STAT628/bestdays_data.csv", index = None)

In [44]:
review_data['date'] = review_data['date'].astype('datetime64[ns]')
review_data['day_of_week'] = review_data['date'].dt.day_name()
review_data['year'] = review_data['date'].dt.year
review_data['hour'] = review_data['date'].dt.hour

In [47]:
del bars_business_data['latitude']
del bars_business_data['longitude']
del bars_business_data['attributes']
del bars_business_data['hours']
del bars_business_data['successful']

In [50]:
bars_business_data = bars_business_data[bars_business_data['state'] != 'AB']

In [53]:
t = bars_business_data.merge(review_data, on = 'business_id', how = 'left')

In [57]:
# t.to_csv("../Documents/STAT628/usmerged_data.csv.gz", index = None, compression = 'gzip')

In [162]:
bars_attribute = pd.read_csv("../Documents/STAT628/barsAttr.csv", index_col = None)

In [164]:
topattrs = ['business_id', 'stars','NoiseLevel', 'hipster', 'HasTV', 'intimate', 'upscale', 'RestaurantsGoodForGroups']

In [165]:
bars_attribute.columns

Index(['business_id', 'stars', 'garage', 'lot', 'street', 'valet', 'validated',
       'BusinessAcceptsCreditCards', 'OutdoorSeating',
       'RestaurantsPriceRange2', 'HasTV', 'casual', 'classy', 'divey',
       'hipster', 'intimate', 'romantic', 'touristy', 'trendy', 'upscale',
       'Alcohol', 'RestaurantsGoodForGroups', 'RestaurantsReservations',
       'RestaurantsTakeOut', 'BikeParking', 'WiFi', 'RestaurantsDelivery',
       'NoiseLevel', 'name', 'city', 'state', 'is_open'],
      dtype='object')

In [191]:
bars_attribute.groupby('hipster').agg({'stars':'mean'})

,stars
hipster,
0,3.686334
1,4.022073


In [199]:
bars_attribute.groupby('RestaurantsGoodForGroups').agg({'stars':'mean'})

,stars
RestaurantsGoodForGroups,
0,3.952977
1,3.619011


In [168]:
bars_attribute[topattrs].to_csv("../Documents/STAT628/topbar_attrs_data.csv", index = None)

In [196]:
t.groupby('state', as_index = False).agg({'review_id' : 'nunique'}).sort_values('review_id', ascending = False)

,state,review_id
11,PA,384609
3,FL,311138
7,LA,191236
12,TN,187308
8,MO,134089
6,IN,130151
0,AZ,90392
1,CA,86093
10,NV,83199
9,NJ,43578


In [180]:
t[t.state.isin(['CA'])].to_csv("../Documents/STAT628/camerged_data.csv.gz", index = None, compression = 'gzip')

In [197]:
t[t.state.isin(['CA'])].to_csv("../Documents/STAT628/camerged_data.csv", index = None)

In [ ]:
t['day_of_review'] = t['date'].astype('datetime64[ns]').dt.day_name()
t['hour_of_review'] = t['date'].astype('datetime64[ns]').dt.hour

In [182]:
t

,business_id,name,address,city,state,postal_code,stars_x,review_count,is_open,categories,review_id,stars_y,text,date,day_of_week,year,hour
0,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,4.0,245,1,"Sushi Bars, Restaurants, Japanese",XYaDbPKyJAu4k2aUOIth5g,5,Stopped in to check out this new spot around t...,2017-12-16 00:13:06,Saturday,2017,0
1,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,4.0,245,1,"Sushi Bars, Restaurants, Japanese",tpLolBuBTx_Ncx3RSf7WBw,1,I live in the neighborhood and used to order a...,2018-04-28 00:46:05,Saturday,2018,0
2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,4.0,245,1,"Sushi Bars, Restaurants, Japanese",u0mLU6WALXLsCM8sNtGxlw,5,We came here tonight just for a date night. We...,2018-01-10 02:56:31,Wednesday,2018,2
3,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,4.0,245,1,"Sushi Bars, Restaurants, Japanese",R5Ha2XF3zevW0D3D6J1d2w,5,Wow! What a great dining adventure! Huge rolls...,2017-12-29 23:57:16,Friday,2017,23
4,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,4.0,245,1,"Sushi Bars, Restaurants, Japanese",6PywmKFi6ma5GfEykq0suw,5,"Wonderfully fresh sushi, amazing lobster fried...",2018-01-22 01:47:25,Monday,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704137,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,4.0,33,1,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",a2vvvqHK29v7IlmgRsufbw,1,They turned off the heaters while we were wait...,2021-11-25 03:35:25,Thursday,2021,3
1704138,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,4.0,33,1,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",E2qgVNEnBo-98acyIXXagQ,5,Very friendly and fast service! We are from o...,2021-05-21 14:20:47,Friday,2021,14
1704139,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,4.0,33,1,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",cbSKIvLa9fKHjtqCs6K9UQ,5,"I used to be a hardcore Starbucks fan, that sa...",2015-05-20 14:44:19,Wednesday,2015,14
1704140,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,4.0,33,1,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",nvJgAjJicW92IdQLGVF3VQ,5,Super friendly service but terrible whiskey. I...,2020-05-17 02:19:36,Sunday,2020,2


In [185]:
bars_attribute[topattrs]

,business_id,stars,NoiseLevel,hipster,HasTV,intimate,upscale,RestaurantsGoodForGroups
0,k0hlBqXX-Bt0vf1op7Jr1w,3.0,1,0,1,0,0,1
1,MUTTqe8uqyMdBl186RmNeA,4.0,1,0,0,0,0,1
2,QdN72BWoyFypdGJhhI5r7g,4.0,1,1,1,0,0,1
3,lk9IwjZXqUMqqOhM774DtQ,3.5,1,0,0,0,0,1
4,txyXRytGjwOXvS8s4sc-WA,3.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
13643,IeSD0nMKRFYUTnR5nZH1CQ,3.5,1,1,1,0,0,1
13644,-R9t30RvsG-Rtiq63yuEmg,2.5,1,0,1,0,0,1
13645,esBGrrmuZzSiECyRBoKvvA,4.5,0,0,1,0,0,1
13646,w_4xUt-1AyY2ZwKtnjW0Xg,4.5,1,0,1,0,0,1


In [198]:
bars_attribute.groupby('RestaurantGoodForGroups').agg({'stars':'mean'})

KeyError: 'RestaurantGoodForGroups'